# SWENSON, HE and Alex
## Coffe Machine Cross-Sell Backend Challenge

In [1]:
import numpy as np
import pandas as pd

In [2]:
def check_item(model, flavor, package):
    '''Checks if combination input matches any item in SKU list, if not returns None'''
    models = {'Small':'CP0', 'Large':'CP1', 'Expresso':'EP0'}
    
    # Check inputs
    if model == 'Expresso':
        flavors = {'vanilla':'0', 'caramel':'1'}
        packages = [3, 5, 7]
        if flavor not in flavors:
            return None
        if package not in packages:
            return None      
    elif model == 'Small' or model == 'Large':
        flavors = {'vanilla':'0', 'caramel':'1', 'psl':'2', 'mocha':'3', 'hazelnut':'4'}
        packages = [1,3]
        if flavor not in flavors:
            return None
        if package not in packages:
            return None      
    else:
        return None
    
    return models[model] + flavors[flavor] + str(package)

def cross_sell(model=[], flavor=[], package=[]):
    '''Returns a list of all cross sell combinations for model, flavor and package'''
    pod_list = []
    
    # Check inputs
    if not model:
        model = ['Small', 'Large', 'Expresso']
    if not flavor:
        flavor = ['vanilla', 'caramel', 'psl', 'mocha', 'hazelnut']
    if not package:
        package = [1, 3, 5, 7]
    
    # All possible combinations
    for f in flavor:
        for p in package: 
            for m in model:
                pod_list.append(check_item(m, f, p))
                
    return pod_list

In [3]:
print(cross_sell(model=['Large'], package=[1]))
print(cross_sell(['Expresso'], ['vanilla']))
print(cross_sell(['Expresso'], package=[3]))

['CP101', 'CP111', 'CP121', 'CP131', 'CP141']
[None, 'EP003', 'EP005', 'EP007']
['EP003', 'EP013', None, None, None]


### Generating Coffe Machine Data Frame

In [4]:
data = np.array("CM001 CM002 CM003 CM101 CM102 CM103 EM001 EM002 EM003".split()).reshape(3,3)
machines = pd.DataFrame(data=data, index="Small Large Expresso".split(), \
             columns="Base Premium Deluxe".split()).transpose()
machines

,Small,Large,Expresso
Base,CM001,CM101,EM001
Premium,CM002,CM102,EM002
Deluxe,CM003,CM103,EM003


### Generating Coffe Pods Data Frame

In [5]:
# Index Levels
outside = ['vanilla']*4+['caramel']*4+['psl']*4+['mocha']*4+['hazelnut']*4
inside = [1,3,5,7]*5
hier_index = list(zip(outside,inside))
hier_index = pd.MultiIndex.from_tuples(hier_index)

data = np.array(cross_sell()).reshape(20, 3) # empty cross_sell function generates pods data 
pods = pd.DataFrame(data=data, index=hier_index, columns="Small Large Expresso".split())
pods.index.names = ['Flavor', 'Packages']
pods

Small  Large Expresso
Flavor   Packages                       
vanilla  1         CP001  CP101     None
         3         CP003  CP103    EP003
         5          None   None    EP005
         7          None   None    EP007
caramel  1         CP011  CP111     None
         3         CP013  CP113    EP013
         5          None   None    EP015
         7          None   None    EP017
psl      1         CP021  CP121     None
         3         CP023  CP123     None
         5          None   None     None
         7          None   None     None
mocha    1         CP031  CP131     None
         3         CP033  CP133     None
         5          None   None     None
         7          None   None     None
hazelnut 1         CP041  CP141     None
         3         CP043  CP143     None
         5          None   None     None
         7          None   None     None

In [6]:
# All large machines
machines['Large']

Base       CM101
Premium    CM102
Deluxe     CM103
Name: Large, dtype: object

In [7]:
# All cross-sell for large machine, smallest per flavor
pods['Large'].xs(1, level='Packages')

Flavor
vanilla     CP101
caramel     CP111
psl         CP121
mocha       CP131
hazelnut    CP141
Name: Large, dtype: object

In [8]:
# All choices on espresso vanilla landing page
pods.Expresso.loc['vanilla'].dropna()

Packages
3    EP003
5    EP005
7    EP007
Name: Expresso, dtype: object

In [9]:
# All espresso machines
machines.Expresso

Base       EM001
Premium    EM002
Deluxe     EM003
Name: Expresso, dtype: object

In [10]:
# All cross-sell for espresso machine, smallest per flavor
pods.Expresso.xs(3, level='Packages').dropna()

Flavor
vanilla    EP003
caramel    EP013
Name: Expresso, dtype: object

In [11]:
# All choices on general vanilla landing page, smallest per product type
pods.loc['vanilla'].loc[1]

Small       CP001
Large       CP101
Expresso     None
Name: 1, dtype: object

In [12]:
# Expresso is missing on the results above, select next smallest value
pods.loc['vanilla'].Expresso[3]

'EP003'

In [13]:
# save data frames for the api
machines.to_csv('machines.csv', encoding='utf-8')
pods.to_csv('pods.csv', encoding='utf-8')